In [2]:
import pandas as pd
%load_ext sql
%sql sqlite:///../datadir/crawl-data.sqlite

'Connected: @../datadir/crawl-data.sqlite'

In [3]:
%%sql 
SELECT *
FROM javascript
LIMIT 5

 * sqlite:///../datadir/crawl-data.sqlite
Done.


id,incognito,browser_id,visit_id,extension_session_uuid,event_ordinal,page_scoped_event_ordinal,window_id,tab_id,frame_id,script_url,script_line,script_col,func_name,script_loc_eval,document_url,top_level_url,call_stack,symbol,operation,value,arguments,time_stamp
1,0,176095192,6501758454862181,517162bc-bbbc-4c10-8b52-5e6f056fcfe2,116,0,1,2,0,http://www.citinet.mn/assets/plugins/swiper/swiper.min.js,16,18972,a.prototype.isSafari<,,http://www.citinet.mn/,http://www.citinet.mn/,,window.navigator.userAgent,get,Mozilla/5.0 (X11; Linux x86_64; rv:105.0) Gecko/20100101 Firefox/105.0,None,2022-11-11T04:17:15.553Z
2,0,176095192,6501758454862181,517162bc-bbbc-4c10-8b52-5e6f056fcfe2,117,1,1,2,0,http://www.citinet.mn/assets/plugins/swiper/swiper.min.js,16,19086,,,http://www.citinet.mn/,http://www.citinet.mn/,,window.navigator.userAgent,get,Mozilla/5.0 (X11; Linux x86_64; rv:105.0) Gecko/20100101 Firefox/105.0,None,2022-11-11T04:17:15.575Z
3,0,176095192,6501758454862181,517162bc-bbbc-4c10-8b52-5e6f056fcfe2,118,2,1,2,0,http://www.citinet.mn/assets/plugins/swiper/swiper.min.js,16,19503,a.prototype.device<,,http://www.citinet.mn/,http://www.citinet.mn/,,window.navigator.userAgent,get,Mozilla/5.0 (X11; Linux x86_64; rv:105.0) Gecko/20100101 Firefox/105.0,None,2022-11-11T04:17:15.576Z
4,0,176095192,6501758454862181,517162bc-bbbc-4c10-8b52-5e6f056fcfe2,119,3,1,2,0,http://www.citinet.mn/assets/plugins/jquery-magnific/jquery.magnific-popup.min.js,4,1209,init,,http://www.citinet.mn/,http://www.citinet.mn/,,window.navigator.appVersion,get,5.0 (X11),None,2022-11-11T04:17:15.576Z
5,0,176095192,6501758454862181,517162bc-bbbc-4c10-8b52-5e6f056fcfe2,120,4,1,2,0,http://www.citinet.mn/assets/plugins/jquery-magnific/jquery.magnific-popup.min.js,4,1419,init,,http://www.citinet.mn/,http://www.citinet.mn/,,window.navigator.userAgent,get,Mozilla/5.0 (X11; Linux x86_64; rv:105.0) Gecko/20100101 Firefox/105.0,None,2022-11-11T04:17:15.576Z


In [4]:
%%sql 
SELECT DISTINCT symbol 
FROM javascript

 * sqlite:///../datadir/crawl-data.sqlite
Done.


symbol
window.navigator.userAgent
window.navigator.appVersion
window.sessionStorage
Storage.setItem
Storage.getItem
Storage.removeItem
window.localStorage
window.document.cookie
window.document.referrer
window.screen.colorDepth


In [5]:
%%sql 
SELECT *
FROM javascript
WHERE symbol = "CanvasRenderingContext2D.getImageData" 
LIMIT 10

 * sqlite:///../datadir/crawl-data.sqlite
Done.


id,incognito,browser_id,visit_id,extension_session_uuid,event_ordinal,page_scoped_event_ordinal,window_id,tab_id,frame_id,script_url,script_line,script_col,func_name,script_loc_eval,document_url,top_level_url,call_stack,symbol,operation,value,arguments,time_stamp
3674,0,176095192,1544802049998897,d26a3187-e0f8-4515-853f-4a7a540d38c5,862,20,1,3,0,https://api-maps.yandex.ru/2.1/?apikey=c6999667-095f-45aa-9077-0c07201c4e59&lang=ru_RU,1,11886,r,,https://www.agatauto.ru/,https://www.agatauto.ru/,,CanvasRenderingContext2D.getImageData,call,,"[49,49,2,2]",2022-11-11T04:21:17.817Z
7274,0,176095192,6289522812095991,863d04f7-f0ba-4b81-834e-bd3640bc61ed,4894,787,1,5,0,https://s3.eu-central-1.amazonaws.com/fi-antifraud-dev.pin-up.dev/fp.js,2,148110,value,,https://www.pin-up409.com/en/,https://www.pin-up409.com/en/,,CanvasRenderingContext2D.getImageData,call,,"[0,0,10,10]",2022-11-11T04:24:00.829Z
7308,0,176095192,6289522812095991,863d04f7-f0ba-4b81-834e-bd3640bc61ed,4928,821,1,5,0,https://s3.eu-central-1.amazonaws.com/fi-antifraud-dev.pin-up.dev/fp.js,2,148110,value,,https://www.pin-up409.com/en/,https://www.pin-up409.com/en/,,CanvasRenderingContext2D.getImageData,call,,"[0,0,10,10]",2022-11-11T04:24:00.838Z
36382,0,176095192,4915088443162928,9b1cbdb0-5453-49b9-82d2-d2cb2faaac85,3709,200,1,10,4294967342,https://cf-assets.hcaptcha.com/c/278beb8b/hsw.js,3,37327,anonymous/_</f</_0x1089a3<,line 1 > Function,https://cf-assets.hcaptcha.com/captcha/v1/0abd424/static/hcaptcha.html#frame=challenge&id=1fbmdtgqe41&host=www.carethy.es&sentry=true&reportapi=https%3A%2F%2Faccounts.hcaptcha.com&recaptchacompat=off&custom=false&endpoint=https%3A%2F%2Fcloudflare.hcaptcha.com&hl=en&assethost=https%3A%2F%2Fcf-assets.hcaptcha.com&imghost=https%3A%2F%2Fcf-imgs.hcaptcha.com&tplinks=on&sitekey=45fbc4de-366c-40ef-9274-9f3feca1cd6c&theme=light&origin=https%3A%2F%2Fwww.carethy.es,https://www.carethy.es/,,CanvasRenderingContext2D.getImageData,call,,"[0,0,8,8]",2022-11-11T04:50:09.972Z
36589,0,176095192,4915088443162928,9b1cbdb0-5453-49b9-82d2-d2cb2faaac85,3916,200,1,10,4294967340,https://cf-assets.hcaptcha.com/c/278beb8b/hsw.js,3,37327,anonymous/_</f</_0x1089a3<,line 1 > Function,https://cf-assets.hcaptcha.com/captcha/v1/0abd424/static/hcaptcha.html#frame=challenge&id=0j46kr75wa89&host=www.carethy.es&sentry=true&reportapi=https%3A%2F%2Faccounts.hcaptcha.com&recaptchacompat=off&custom=false&endpoint=https%3A%2F%2Fcloudflare.hcaptcha.com&hl=en&assethost=https%3A%2F%2Fcf-assets.hcaptcha.com&imghost=https%3A%2F%2Fcf-imgs.hcaptcha.com&tplinks=on&sitekey=45fbc4de-366c-40ef-9274-9f3feca1cd6c&theme=light&origin=https%3A%2F%2Fwww.carethy.es,https://www.carethy.es/,,CanvasRenderingContext2D.getImageData,call,,"[0,0,8,8]",2022-11-11T04:50:10.051Z
41798,0,176095192,4462946572504436,a327f6cb-f556-438c-89c3-ecabab6290bb,425,230,1,1,0,https://www.sophos.com/akam/13/8875cc5,2,1344,a/A/t[_[249]],,https://www.sophos.com/en-us,https://www.sophos.com/en-us,,CanvasRenderingContext2D.getImageData,call,,"[0,0,1,1]",2022-11-11T04:54:35.741Z
60308,0,176095192,6465969384105208,3c2424e7-066b-4fcf-a7d7-38fd0c171cd5,526,318,1,1,0,https://embed.dugout.com/v3.1/wpmedia.js,38,53148,p,,https://sportowefakty.wp.pl/,https://sportowefakty.wp.pl/,,CanvasRenderingContext2D.getImageData,call,,"[0,0,1,1]",2022-11-11T05:16:41.107Z
74473,0,176095192,6759835249162585,fe062aac-b238-470d-bb01-56530585f2cc,3043,364,1,5,0,https://cds-sdkcfg.onlineaccess1.com/common.js,1,84438,Bb<,,https://bell.bank/,https://bell.bank/,,CanvasRenderingContext2D.getImageData,call,,"[0,0,5,5]",2022-11-11T05:29:49.453Z
74479,0,176095192,6759835249162585,fe062aac-b238-470d-bb01-56530585f2cc,3049,370,1,5,0,https://cds-sdkcfg.onlineaccess1.com/common.js,1,84438,Bb<,,https://bell.bank/,https://bell.bank/,,CanvasRenderingContext2D.getImageData,call,,"[0,0,5,5]",2022-11-11T05:29:49.456Z
74486,0,176095192,6759835249162585,fe062aac-b238-470d-bb01-56530585f2cc,3056,377,1,5,0,https://cds-sdkcfg.onlineaccess1.com/common.js,1,84438,Bb<,,https:

In [6]:
import ast

def Canvas(df :  pd.DataFrame) -> bool:
    heightRec = False
    widthRec = False
    colors = set()
    charecters = set()
    Extraction = False
    for row in df.itertuples():
        match row.symbol:
            case 'HTMLCanvasElement.height':
                if row.operation == 'set' and row.value and int(row.value) > 16:
                    heightRec = True
            case 'HTMLCanvasElement.width':
                if row.operation == 'set' and row.value and int(row.value) > 16:
                    widthRec = True
            case 'CanvasRenderingContext2D.fillText':
                args = ast.literal_eval(row.arguments)
                for char in args[0]:
                    charecters.add(char)
            case 'CanvasRenderingContext2D.fillStyle':
                if row.operation == 'set' and row.value:
                    colors.add(row.value)
            case 'HTMLCanvasElement.toDataURL':
                Extraction = True
            case 'CanvasRenderingContext2D.getImageData':
                Extraction = True
    return heightRec and widthRec and ( len(colors) > 2 or len(charecters) > 10) and Extraction

# functions = [ ("Canvas", Canvas ) ]

In [9]:
script_urls = %sql SELECT DISTINCT script_url FROM javascript LIMIT 5000
canvasFingerprinters = []
for script in script_urls:
    script : str = script[0]
    result = %sql SELECT * FROM javascript WHERE script_url = '{script}'
    if result:
        df : pd.DataFrame = result.DataFrame()
        if Canvas(df):
            canvasFingerprinters.append(script)
    

 * sqlite:///../datadir/crawl-data.sqlite
Done.
 * sqlite:///../datadir/crawl-data.sqlite
Done.
 * sqlite:///../datadir/crawl-data.sqlite
Done.
 * sqlite:///../datadir/crawl-data.sqlite
Done.
 * sqlite:///../datadir/crawl-data.sqlite
Done.
 * sqlite:///../datadir/crawl-data.sqlite
Done.
 * sqlite:///../datadir/crawl-data.sqlite
Done.
 * sqlite:///../datadir/crawl-data.sqlite
Done.
 * sqlite:///../datadir/crawl-data.sqlite
Done.
 * sqlite:///../datadir/crawl-data.sqlite
Done.
 * sqlite:///../datadir/crawl-data.sqlite
Done.
 * sqlite:///../datadir/crawl-data.sqlite
Done.
 * sqlite:///../datadir/crawl-data.sqlite
Done.
 * sqlite:///../datadir/crawl-data.sqlite
Done.
 * sqlite:///../datadir/crawl-data.sqlite
Done.
 * sqlite:///../datadir/crawl-data.sqlite
Done.
 * sqlite:///../datadir/crawl-data.sqlite
Done.
 * sqlite:///../datadir/crawl-data.sqlite
Done.
 * sqlite:///../datadir/crawl-data.sqlite
Done.
 * sqlite:///../datadir/crawl-data.sqlite
Done.
 * sqlite:///../datadir/crawl-data.sqlit

ValueError: invalid literal for int() with base 10: '223.9219012998221'

In [ ]:
canvasFingerprinters

[]